In [ ]:
from netCDF4 import Dataset                     # For reading data
from matplotlib.colors import LogNorm                 # For plotting
from matplotlib.patches import Rectangle
import numpy.ma as ma
from mpl_toolkits.basemap import Basemap,shiftgrid
import matplotlib.pyplot as plt
import matplotlib.colors as cols
from matplotlib.colors import BoundaryNorm
from matplotlib.colors import from_levels_and_colors
import matplotlib as mpl
import xray as xr
import time
from mpas_xarray import preprocess_mpas_timeSeriesStats, remove_repeated_time_index
# Place figures within document
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (8.0, 8.0) # Large figures
#plotting rules
axis_font = {'fontname':'Arial', 'size':'16'}    
title_font = {'fontname':'Arial', 'size':'20', 'color':'black', 'weight':'normal'}

In [ ]:
yr_offset = 1970
climo_yr1 = 40   # start year of climatology
climo_yr2 = 49  # end model year of climatology
figure_dir = '/scratch2/scratchdirs/lvroekel/1850/' #place to save figures
figure_tag = '.png' #type of figure to create
climo_yrTag = 'yr%d-%d_avg'%(climo_yr1,climo_yr2)
infile_path = '/scratch1/scratchdirs/golaz/ACME_simulations/20160520.A_WCYCL1850.ne30_oEC.edison.alpha6_01/' + \
                'run/am.mpas-o.timeSeriesStats.004*.nc'
outputTimes = [1,6,12,14,16] #times to output (0-11 = Jan - Dec, 12 - JFM, 13 - AMJ, 14-JAS, 15-OND, 16 - Annual)
MLD_DATA = 'holtetalley'
MLDclevs = [0, 20, 40, 60, 80, 100, 150, 200, 400, 800] #Chosen color levels (needs to be 7) for raw MLD plots
#MLDclevs = np.linspace(0,800,10)
BIASclevs = [-175, -125, -75, -25, 25, 75, 125, 175] #Chosen color levels (needs to be 7) for bias MLD plots

In [ ]:
print 'yr%d-%d_avg'%(climo_yr1,climo_yr2)

In [ ]:
#A case
try:
    ds = xr.open_mfdataset(infile_path,preprocess=lambda x: preprocess_mpas_timeSeriesStats(x, yearoffset=yr_offset, \
                        timestr='timeSeriesStats_avg_daysSinceStartOfSim',                          \
                        onlyvars=['time_avg_tThreshMLD',                                \
                                  'time_avg_dThreshMLD','latCell','lonCell']))
    ds = remove_repeated_time_index(ds)
except: #use old xray wrapper if timeSeriesSTats_avg_daysSinceStartOfSim not available
    yr_offset=1700
    from mpas_xray import preprocess_mpas, remove_repeated_time_index

# open up series of netcdf files and use the preprocess_mpas to get viable times
    ds = xr.open_mfdataset(infile_path, preprocess=preprocess_mpas)
    ds = remove_repeated_time_index(ds)

In [ ]:
#Load the comparison data

nLon = 360
nLat = 180
monMLDTemp = np.zeros((17,nLon,nLat))
monMLDDen = np.zeros((17,nLon,nLat))

if MLD_DATA.lower() == 'jamstec':
    for i in range(12):
        filename='/usr/projects/climate/SHARED_CLIMATE/observations/MLD/MLD_JAMSTEC/0.2deltaT/2x2/ml_MLD_CLIM_%02d.nc'%(i+1)
        temp = xr.open_mfdataset(filename)
        monMLDTemp[i,:,:]=temp.MLD.values[:,:].T

        filename='/usr/projects/climate/SHARED_CLIMATE/observations/MLD/MLD_JAMSTEC/0.03sigmaTheta/2x2/ml_MLD_CLIM_%02d.nc'%(i+1)
        temp = xr.open_mfdataset(filename)
        monMLDDen[i,:,:]=temp.MLD.values[:,:].T

    latDATA = temp.LATITUDE.values[:]
    lonDATA = temp.LONGITUDE.values[:]
    
elif MLD_DATA.lower() == 'holtetalley':
    filename = '/scratch2/scratchdirs/lvroekel/holtetalley_mld_climatology.nc'
    temp = xr.open_mfdataset(filename)
    #This dataset does not have an explicit temperature MLD, just fill with same density value
    monMLDTemp[:12,:,:] = temp.mld_da_mean[:,:,:].values.T
    monMLDDen[:12,:,:] = temp.mld_da_mean[:,:,:].values.T

    latDATA = temp.lat.values[:]
    lonDATA = temp.lon.values[:]
    
else:
    raise ValueError('Choice of MLD Dataset not supported, currently JAMSTEC or HOLTETALLEY only')
#Get MLD grid latitude and longitude

In [ ]:
yr_offset=1700

In [ ]:
# Compute climatologies (first motnhly and then seasonally)
time_start = datetime.datetime(yr_offset+climo_yr1,1,1)
time_end = datetime.datetime(yr_offset+climo_yr2,12,31)
ds_tslice = ds.sel(Time=slice(time_start,time_end))
monthly_clim = ds_tslice.groupby('Time.month').mean('Time')
[djan,dfeb,dmar,dapr,dmay,djun,djul,daug,dsep,doct,dnov,ddec] = [31,28,31,30,31,30,31,31,30,31,30,31]
JFM = (djan*monthly_clim.sel(month=1) + dfeb*monthly_clim.sel(month=2) + dmar*monthly_clim.sel(month=3)) / (djan + dfeb + dmar)
AMJ = (dapr*monthly_clim.sel(month=4) + dmay*monthly_clim.sel(month=5) + djun*monthly_clim.sel(month=6)) / (dapr + dmay + djun)
JAS = (djul*monthly_clim.sel(month=7) + daug*monthly_clim.sel(month=8) + dsep*monthly_clim.sel(month=9)) / (djul + daug + dsep)
OND = (doct*monthly_clim.sel(month=10) + dnov*monthly_clim.sel(month=11) + ddec*monthly_clim.sel(month=12)) / (doct + dnov + ddec)
ANN = monthly_clim.mean(dim='month')

monMLDTemp[12,:,:] = (djan*monMLDTemp[0,:,:]+dfeb*monMLDTemp[1,:,:]+dmar*monMLDTemp[2,:,:]) / (djan + dfeb + dmar)
monMLDDen[12,:,:] = (djan*monMLDDen[0,:,:]+dfeb*monMLDDen[1,:,:]+dmar*monMLDDen[2,:,:]) / (djan + dfeb + dmar)
monMLDTemp[13,:,:] = (dapr*monMLDTemp[3,:,:]+dmay*monMLDTemp[4,:,:]+djun*monMLDTemp[5,:,:]) / (dapr + dmay + djun)
monMLDDen[13,:,:] = (dapr*monMLDDen[3,:,:]+dmay*monMLDDen[4,:,:]+djun*monMLDDen[5,:,:]) / (dapr + dmay + djun)
monMLDTemp[14,:,:] = (djul*monMLDTemp[6,:,:]+daug*monMLDTemp[7,:,:]+dsep*monMLDTemp[8,:,:]) / (djul + daug + dsep)
monMLDDen[14,:,:] = (djul*monMLDDen[6,:,:]+daug*monMLDDen[7,:,:]+dsep*monMLDDen[8,:,:]) / (djul + daug + dsep)
monMLDTemp[15,:,:] = (doct*monMLDTemp[9,:,:]+dnov*monMLDTemp[10,:,:]+ddec*monMLDTemp[11,:,:]) / (dnov + doct + ddec)
monMLDDen[15,:,:] = (doct*monMLDDen[9,:,:]+dnov*monMLDDen[10,:,:]+ddec*monMLDDen[11,:,:]) / (doct + dnov + ddec)
monMLDTemp[16,:,:] = monMLDTemp.mean(axis=0)
monMLDDen[16,:,:] = monMLDDen.mean(axis=0)

In [ ]:
from scipy.spatial import cKDTree

def lon_lat_to_cartesian(lon, lat, R = 6371222):
    """
    calculates lon, lat coordinates of a point on a sphere with
    radius R
    """
    lon_r = np.radians(lon)
    lat_r = np.radians(lat)

    x = R * np.cos(lat_r) * np.cos(lon_r)
    y = R * np.cos(lat_r) * np.sin(lon_r)
    z = R * np.sin(lat_r)
    return x,y,z

def init_tree(lonCell, latCell):
    
#   lonCell[lonCell > np.pi] -= 2.0*np.pi
    rtd = 180.0 / np.pi
    xs, ys, zs = lon_lat_to_cartesian(lonCell*rtd,latCell*rtd)
    tree = cKDTree(zip(xs, ys, zs))

    if min(lonDATA) < 0:
        indvals = np.where(lonDATA < 0)
        lonDATA[indvals] += 360
    latTarg, lonTarg = np.meshgrid(latDATA,lonDATA)
    xt, yt, zt = lon_lat_to_cartesian(lonTarg.flatten(),latTarg.flatten())
    
    d, inds = tree.query(zip(xt, yt, zt), k = 1)
    
    return d, inds, lonTarg, latTarg

def interp_fields(field, d, inds):
    #d, inds = tree.query(zip(xt, yt, zt), k = 10)
    #w = 1.0 / d**2
    #print d.shape
    #interpFld = np.sum(w * field.flatten()[inds],axis=1) / np.sum(w,axis=1)
    #interpFld.shape = lonTarg.shape
    
    #return interpFld
    return field.flatten()[inds].reshape(lonTarg.shape)
     

In [ ]:
#interpolate MPAS data for months on to ARGO grid Include seasons here
MPAS_MLD_Temp = np.zeros((17,nLon,nLat))
MPAS_MLD_Den = np.zeros((17,nLon,nLat))
d, inds, lonTarg, latTarg = init_tree(ds.lonCell.values[0,:],ds.latCell.values[0,:])

MLD_BIAS_Temp = np.zeros((17,nLon,nLat))
MLD_BIAS_Den = np.zeros((17,nLon,nLat))

for i in range(12):
    cur_mon = monthly_clim.sel(month=i+1)
    MPAS_MLD_Temp[i,:,:] = interp_fields(cur_mon.time_avg_tThreshMLD.values[:], d, inds)
    MPAS_MLD_Den[i,:,:] = interp_fields(cur_mon.time_avg_dThreshMLD.values[:], d, inds)
    
    MLD_BIAS_Temp[i,:,:] = MPAS_MLD_Temp[i,:,:] - monMLDTemp[i,:,:]
    MLD_BIAS_Den[i,:,:] = MPAS_MLD_Den[i,:,:] - monMLDDen[i,:,:]
    
#Season average

MPAS_MLD_Temp[12,:,:] = interp_fields(JFM.time_avg_tThreshMLD.values[:], d, inds)
MPAS_MLD_Den[12,:,:] = interp_fields(JFM.time_avg_dThreshMLD.values[:], d, inds)

MLD_BIAS_Temp[12,:,:] = MPAS_MLD_Temp[12,:,:] - monMLDTemp[12,:,:]
MLD_BIAS_Den[12,:,:] = MPAS_MLD_Den[12,:,:] - monMLDDen[12,:,:]

MPAS_MLD_Temp[13,:,:] = interp_fields(AMJ.time_avg_tThreshMLD.values[:], d, inds)
MPAS_MLD_Den[13,:,:] = interp_fields(AMJ.time_avg_dThreshMLD.values[:], d, inds)

MLD_BIAS_Temp[13,:,:] = MPAS_MLD_Temp[13,:,:] - monMLDTemp[13,:,:]
MLD_BIAS_Den[13,:,:] = MPAS_MLD_Den[13,:,:] - monMLDDen[13,:,:]

MPAS_MLD_Temp[14,:,:] = interp_fields(JAS.time_avg_tThreshMLD.values[:], d, inds)
MPAS_MLD_Den[14,:,:] = interp_fields(JAS.time_avg_dThreshMLD.values[:], d, inds)

MLD_BIAS_Temp[14,:,:] = MPAS_MLD_Temp[14,:,:] - monMLDTemp[14,:,:]
MLD_BIAS_Den[14,:,:] = MPAS_MLD_Den[14,:,:] - monMLDDen[14,:,:]

MPAS_MLD_Temp[15,:,:] = interp_fields(OND.time_avg_tThreshMLD.values[:], d, inds)
MPAS_MLD_Den[15,:,:] = interp_fields(OND.time_avg_dThreshMLD.values[:], d, inds)

MLD_BIAS_Temp[15,:,:] = MPAS_MLD_Temp[15,:,:] - monMLDTemp[15,:,:]
MLD_BIAS_Den[15,:,:] = MPAS_MLD_Den[15,:,:] - monMLDTemp[15,:,:]

MPAS_MLD_Temp[16,:,:] = interp_fields(ANN.time_avg_tThreshMLD.values[:], d, inds)
MPAS_MLD_Den[16,:,:] = interp_fields(ANN.time_avg_dThreshMLD.values[:], d, inds)

MLD_BIAS_Temp[16,:,:] = MPAS_MLD_Temp[16,:,:] - monMLDTemp[16,:,:]
MLD_BIAS_Den[16,:,:] = MPAS_MLD_Den[16,:,:] - monMLDDen[16,:,:]


In [ ]:
figure_labels=['January','February','March','April','May','June','July','August','September','October','November'\
              ,'December','JFM','AMJ','JAS','OND','Annual']

In [ ]:
#Plot MLDs based on temperature and desnity threshold criteria

#lonTargN = np.roll(lonTarg,180,axis=0)
#latTargN = np.roll(latTarg,180,axis=0)
fig=plt.figure(figsize=(8,11),dpi=300)

if max(lonDATA) > 180:
    indvals = np.where(lonDATA > 180)
    lonDATA[indvals] -= 360
latTarg, lonTarg = np.meshgrid(latDATA,lonDATA)

for label in ['Temp','Den']: #loop over temperature and density threshold criteria for MLD

#    exec('MPAS_MLD = np.roll(MPAS_MLD_'+label+',180,axis=1)')
#    exec('monMLD = np.roll(monMLD'+label+',180,axis=1)')
#    exec('MLD_BIAS = np.roll(MLD_BIAS_'+label+',180,axis=1)')

    exec('MPAS_MLD = MPAS_MLD_'+label)
    exec('monMLD = monMLD'+label)
    exec('MLD_BIAS = MLD_BIAS_'+label)
    
    for i in outputTimes:
        ax = fig.add_subplot(311)
        m = Basemap(projection='cyl',llcrnrlat=-85,urcrnrlat=85,\
            llcrnrlon=-180,urcrnrlon=180,resolution='l')
        m.drawcoastlines()
        m.fillcontinents(color='grey',lake_color='white')
        m.drawparallels(np.arange(-90.,120.,30.))
        m.drawmeridians(np.arange(0.,360.,60.),labels=[False,False,False,True])
        #temp,lonOut = shiftgrid(180.,np.squeeze(MPAS_MLD_Den[i,:,:]),lonTarg)
        x, y = m(lonTarg, latTarg) # compute map proj coordinates
        nice_cmap = plt.get_cmap('viridis')
        lev_cmap = nice_cmap([20,80,110,140,170,200,230,235,240])
        new_cmap = cols.ListedColormap(lev_cmap,"mv_cmap")
        
        norm = mpl.colors.BoundaryNorm(MLDclevs, new_cmap.N)

        cs = m.contourf(x,y,MPAS_MLD[i,:,:],cmap=new_cmap,norm=norm,spacing='uniform',levels=MLDclevs)
        #cs = m.contourf(x,y,MLD_BIAS_Den[i,:,:],spacing='uniform',levels=clevs)
        cbar = m.colorbar(cs,location='right',pad="15%",spacing='uniform',extendfrac='auto',\
                          extendrect='True',ticks=MLDclevs, boundaries=MLDclevs)
        cbar.set_label('Meters')
        plt.title('ACMEv1',y=1.04)
        ax = fig.add_subplot(312)

        m = Basemap(projection='cyl',llcrnrlat=-85,urcrnrlat=85,\
            llcrnrlon=-180,urcrnrlon=180,resolution='l')
        m.drawcoastlines()
        m.fillcontinents(color='grey',lake_color='white')
        m.drawparallels(np.arange(-90.,120.,30.))
        m.drawmeridians(np.arange(0.,360.,60.),labels=[False,False,False,True])
        x, y = m(lonTarg, latTarg) # compute map proj coordinates
        nice_cmap = plt.get_cmap('viridis')
        lev_cmap = nice_cmap([20, 80, 110, 140, 170, 200, 230, 235, 240])
        new_cmap = cols.ListedColormap(lev_cmap,"mv_cmap")

        norm = mpl.colors.BoundaryNorm(MLDclevs, new_cmap.N)
        cs = m.contourf(x, y, monMLD[i,:,:], cmap = new_cmap, norm = norm, spacing = 'uniform', levels = MLDclevs)
        #cs = m.contourf(x,y,MLD_BIAS_Den[i,:,:],spacing='uniform',levels=clevs)
        cbar = m.colorbar(cs,location='right',pad="15%",spacing='uniform',extendfrac='auto',\
                          extendrect='True',ticks=MLDclevs, boundaries=MLDclevs)
        cbar.set_label('Meters')
        plt.title('ARGO MLD',y=1.04)
        ax = fig.add_subplot(313)

        m = Basemap(projection='cyl',llcrnrlat=-85,urcrnrlat=85,\
            llcrnrlon=-180,urcrnrlon=180,resolution='l')
        m.drawcoastlines()
        m.fillcontinents(color='grey',lake_color='white')
        m.drawparallels(np.arange(-90.,120.,30.))
        m.drawmeridians(np.arange(0.,360.,60.),labels=[False,False,False,True])
        x, y = m(lonTarg, latTarg) # compute map proj coordinates
        nice_cmap = plt.get_cmap('RdYlBu_r')
        lev_cmap = nice_cmap([20,80,110,140,170,200,230,235,240])
        new_cmap = cols.ListedColormap(lev_cmap,"mv_cmap")
    
        norm = mpl.colors.BoundaryNorm(BIASclevs, new_cmap.N)
        cs = m.contourf(x, y, MLD_BIAS[i,:,:], cmap = new_cmap, norm = norm, spacing = 'uniform', levels = BIASclevs)
        #cs = m.contourf(x,y,MLD_BIAS_Den[i,:,:],spacing='uniform',levels=clevs)
        plt.title('ACMEv1 - OBS',y=1.04)
        cbar = m.colorbar(cs,location='right',pad="15%",spacing='uniform',extendfrac='auto',\
                          extendrect='True',ticks=BIASclevs, boundaries=BIASclevs)
        cbar.set_label('Meters')
        #fig.suptitle(figure_labels[i]+' Mixed Layer Depth Bias',y=.98)
        fig.tight_layout()
        #fig.savefig('/Volumes/X/CoupledACME/'+figure_labels[i]+'_mld_bias_Acase.png')
        fig.savefig(figure_dir+figure_labels[i]+'_mld_bias_'+label+'_year_'+climo_yrTag+'_criterion'+figure_tag)

In [ ]:
#Plot MLDs based on temperature and desnity threshold criteria

lonTargN = np.roll(lonTarg,180,axis=0)
latTargN = np.roll(latTarg,180,axis=0)
fig=plt.figure(figsize=(8,11),dpi=300)

for label in ['Temp','Den']: #loop over temperature and density threshold criteria for MLD

    exec('MPAS_MLD = np.roll(MPAS_MLD_'+label+',180,axis=1)')
    exec('monMLD = np.roll(monMLD'+label+',180,axis=1)')
    exec('MLD_BIAS = np.roll(MLD_BIAS_'+label+',180,axis=1)')

    for i in outputTimes:
        ax = fig.add_subplot(311)
        m = Basemap(projection='robin',lon_0=0,resolution='l')
        m.drawcoastlines()
        m.fillcontinents(color='grey',lake_color='white')
        m.drawparallels(np.arange(-90.,120.,30.))
        m.drawmeridians(np.arange(0.,360.,60.),labels=[False,False,False,True])
        #temp,lonOut = shiftgrid(180.,np.squeeze(MPAS_MLD_Den[i,:,:]),lonTarg)
        x, y = m(lonTargN, latTargN) # compute map proj coordinates
        nice_cmap = plt.get_cmap('viridis')
        lev_cmap = nice_cmap([20,80,110,140,170,200,230,235,240])
        new_cmap = cols.ListedColormap(lev_cmap,"mv_cmap")
        
        norm = mpl.colors.BoundaryNorm(MLDclevs, new_cmap.N)

        cs = m.contourf(x,y,MPAS_MLD[i,:,:],cmap=new_cmap,norm=norm,spacing='uniform',levels=MLDclevs)
        #cs = m.contourf(x,y,MLD_BIAS_Den[i,:,:],spacing='uniform',levels=clevs)
        cbar = m.colorbar(cs,location='right',pad="15%",spacing='uniform',extendfrac='auto',\
                          extendrect='True',ticks=MLDclevs, boundaries=MLDclevs)
        cbar.set_label('Meters')
        plt.title('ACMEv1',y=1.04)
        ax = fig.add_subplot(312)

        m = Basemap(projection='robin',lon_0=0,resolution='l')
        m.drawcoastlines()
        m.fillcontinents(color='grey',lake_color='white')
        m.drawparallels(np.arange(-90.,120.,30.))
        m.drawmeridians(np.arange(0.,360.,60.),labels=[False,False,False,True])
        x, y = m(lonTargN, latTargN) # compute map proj coordinates
        nice_cmap = plt.get_cmap('viridis')
        lev_cmap = nice_cmap([20, 80, 110, 140, 170, 200, 230, 235, 240])
        new_cmap = cols.ListedColormap(lev_cmap,"mv_cmap")

        norm = mpl.colors.BoundaryNorm(MLDclevs, new_cmap.N)
        cs = m.contourf(x, y, monMLD[i,:,:], cmap = new_cmap, norm = norm, spacing = 'uniform', levels = MLDclevs)
        #cs = m.contourf(x,y,MLD_BIAS_Den[i,:,:],spacing='uniform',levels=clevs)
        cbar = m.colorbar(cs,location='right',pad="15%",spacing='uniform',extendfrac='auto',\
                          extendrect='True',ticks=MLDclevs, boundaries=MLDclevs)
        cbar.set_label('Meters')
        plt.title('ARGO MLD',y=1.04)
        ax = fig.add_subplot(313)

        m = Basemap(projection='robin',lon_0=0,resolution='l')
        m.drawcoastlines()
        m.fillcontinents(color='grey',lake_color='white')
        m.drawparallels(np.arange(-90.,120.,30.))
        m.drawmeridians(np.arange(0.,360.,60.),labels=[False,False,False,True])
        x, y = m(lonTargN, latTargN) # compute map proj coordinates
        nice_cmap = plt.get_cmap('RdYlBu_r')
        lev_cmap = nice_cmap([20,80,110,140,170,200,230,235,240])
        new_cmap = cols.ListedColormap(lev_cmap,"mv_cmap")
    
        norm = mpl.colors.BoundaryNorm(BIASclevs, new_cmap.N)
        cs = m.contourf(x, y, MLD_BIAS[i,:,:], cmap = new_cmap, norm = norm, spacing = 'uniform', levels = BIASclevs)
        #cs = m.contourf(x,y,MLD_BIAS_Den[i,:,:],spacing='uniform',levels=clevs)
        plt.title('ACMEv1 - OBS',y=1.04)
        cbar = m.colorbar(cs,location='right',pad="15%",spacing='uniform',extendfrac='auto',\
                          extendrect='True',ticks=BIASclevs, boundaries=BIASclevs)
        cbar.set_label('Meters')
        #fig.suptitle(figure_labels[i]+' Mixed Layer Depth Bias',y=.98)
        fig.tight_layout()
        #fig.savefig('/Volumes/X/CoupledACME/'+figure_labels[i]+'_mld_bias_Acase.png')
        fig.savefig(figure_dir+figure_labels[i]+'_mld_bias_'+label+'_criterion'+figure_tag)